# 📘 Example: Creating a Hurricane Event in FloodAdapt

This notebook demonstrates how to create a hurricane event using FloodAdapt. Hurricane events are valuable for controlled testing, sensitivity analysis, and understanding the behavior of flood models under simplified or hypothetical scenarios. 

A FloodAdapt Event consists of 2 things:

- a [`TimeFrame`](../../api_ref/TimeFrame.qmd) describing the start and end time of the hazard simulation(s)
- a collection of forcings to be applied to the hazard model(s)

In this example, we construct a full `HurricaneEvent` with `water level`, `rainfall`, `wind`, and river `discharge` forcings, and then save it to a FloodAdapt database.

## ⏱️ Step 1. Setup and Imports

We begin by importing the required classes and modules for constructing hurricane forcings and managing event data within the flood_adapt framework.

In [ ]:
import flood_adapt.objects.forcing as f

from pathlib import Path
from datetime import datetime

from flood_adapt.objects import HurricaneEvent, TimeFrame
from flood_adapt.objects.events.hurricane import TranslationModel
from flood_adapt import unit_system as us
from flood_adapt import FloodAdapt, Settings

# Setup FloodAdapt
DATA_DIR = Path("../../../_data").resolve()
STATIC_DATA_DIR = DATA_DIR / "examples" / "static-data"
settings = Settings(
    DATABASE_ROOT=DATA_DIR / "examples",
    DATABASE_NAME="charleston_test"
)
fa = FloodAdapt(database_path=settings.database_path)

## 🗓️ Step 2. Define the Simulation Time Frame

We specify a one-day time frame for the hurricane event, from January 1 to January 2, 2025. Make sure the time frame covers the tinme specified in your hurricane track file. 

In [ ]:
# Create an time frame for the simulation
start_time = datetime(year=2025, month=1, day=1)
end_time = datetime(year=2025, month=1, day=2)
time_frame = TimeFrame(start_time=start_time, end_time=end_time)

## 🌊 Step 3. Define Water Level Forcing

Water levels for Hurricane Events are computed by taking the Hurricane Track, and generating a pressure and wind field along its track.

These fields are then used as forcing inputs to the offshore simulation, which generates the storm surge to be used for the overland simulation. So for waterlevels, we only need to specify to use the offshore model as the input, denoted with `WaterlevelModel`

In [ ]:
water_levels = f.WaterlevelModel()

## 🌧️ Step 3. Obtain a hurricane track

You can include an [IBTrACS](https://www.ncei.noaa.gov/products/international-best-track-archive) hurricane database with recent and historic tracks in the FloodAdapt database. You can use any track from the database and optionally shift the track north, southh, east or west. 

>**NOTE:** Hurricane events are only available in FloodAdapt  when you have an offshore flood hazard model in your database that simulates the surge from the hurricane track. 


In [ ]:
# Get the cyclone database
cyclone_db = fa.database.static.get_cyclone_track_database()
ian_index = cyclone_db.list_names().index("IAN")

# Not all cyclone tracks have names, in addition to duplicate names existing, so the index is required
track = fa.get_cyclone_track_by_index(index=ian_index) 
track_file = STATIC_DATA_DIR / "IAN.cyc"
track.write_track(filename=track_file, fmt="ddb_cyc")

# Optionally translate the cyclone track from what is defined in the file
translation = TranslationModel(
    eastwest_translation=us.UnitfulLength(value=3000, units=us.UnitTypesLength.meters),
    northsouth_translation=us.UnitfulLength(value=5000, units=us.UnitTypesLength.meters),
)

## 🌧️ Step 4. Define Track forcings

Given a Hurricane track, the wind field is always computer from the track. You can choose to model the rainfall based on a parametric model around the hurricane track or choose any iother rainfall option. 

In [ ]:
# We want to include the rainfall and wind from the hurricane track
rainfall = f.RainfallTrack(path=track_file)
wind = f.WindTrack(path=track_file)

## 🏞️ Step 5. Define River Discharge Forcing

Discharge is defined for two pre-configured rivers in this example. These rivers must be registered in the hazard model configuration beforehand, see [Database builder].

In [ ]:
# The available rivers are defined in the hazard model when creating the database.
# You cannot add new rivers to the model in an event
# You can only set the discharge of each given river.
river = fa.database.site.sfincs.river[0]

discharge = f.DischargeConstant(
    river=river,
    discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms)
)

# Inspect
df = discharge.to_dataframe(time_frame=time_frame)
df.plot(title="Constant Discharge River", xlabel="Time", ylabel="Discharge (cms)", legend=True, figsize=(5, 2))

## 🧩 Step 6. Combine Forcings and Create hurricane Event

All defined forcings are collected into a single dictionary, which is used to construct a hurricaneEvent.

> **NOTE**: each event can only have 1 forcing of the types: `water level`, `rainfall` and `wind`. For `discharge` however, each river is required to have a forcing associated with it.

In [ ]:
# Create a hurricaneEvent with the forcings and time frame
event = HurricaneEvent(
    name="example_hurricane_event",
    time=time_frame,
    forcings = {
        f.ForcingType.WATERLEVEL: [water_levels],
        f.ForcingType.RAINFALL: [rainfall],
        f.ForcingType.WIND: [wind],
        f.ForcingType.DISCHARGE: [discharge],
    },
    track_name=track.name,
    hurricane_translation=translation,
)

## 💾 Step 7. Save the Event to a FloodAdapt Database

Finally, we save the event to a FloodAdapt database.

In [ ]:
# Save the event to the database
fa.save_event(event=event)